In [1]:
import os
import torch
import torch.nn as nn
import torchtext.data as data
import torchtext.datasets as datasets
import torch.nn.functional as F
from torchtext.vocab import Vectors 
from torchtext.data import TabularDataset
import torch.optim as optim

import time

from nltk.tokenize import word_tokenize

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import random

import pandas as pd

from transformers import BertTokenizer, BertModel

## tokenizer 생성

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
len(tokenizer.vocab)

30522

In [3]:
sample = "hello world"
sample_token = tokenizer.tokenize(sample)
print(sample_token)

['hello', 'world']


In [4]:
sample_index = tokenizer.convert_tokens_to_ids(sample_token)
print(sample_index)

[7592, 2088]


In [5]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [6]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [7]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [8]:
max_input_length = 128
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

## Field 생성

In [9]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [10]:
train_data, test_data = TabularDataset.splits(path =".", train = "./train_test_data/train_data_fin.csv", test="./train_test_data/test_data_fin.csv", 
                        format="csv", fields=[("text", TEXT), ("label", LABEL)], skip_header=True)

In [11]:
train_data, valid_data = train_data.split(split_ratio=0.8, random_state =random.seed(0) )

In [12]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 6090
Number of validation examples: 1523
Number of testing examples: 3263


In [13]:
print(vars(train_data[0]))
print(vars(train_data[1]))
print(vars(test_data[17]))

{'text': [10716, 2067, 2039], 'label': '0.0'}
{'text': [1996, 2159, 1997, 1996, 3842, 23713, 12368, 5680, 2451, 2024, 2006, 7397, 17712, 3995, 26493, 8591, 26965, 19222, 12259, 5057, 17698, 4047, 4181, 11475, 12155, 20899], 'label': '0.0'}
{'text': [3130, 15422, 6977, 18447, 2615, 2000, 3489, 5003, 4939, 4213, 2226, 29653, 2015, 3510, 5325, 4520, 11884, 10474, 11113, 24472], 'label': '0'}


In [14]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data[0])["text"])
print(tokens)

['noel', 'back', 'up']


In [15]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'0.0': 0, '1.0': 1})


In [16]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator  = data.BucketIterator.splits((train_data, valid_data),
                                                            batch_size = BATCH_SIZE, shuffle = True, sort=False,
                                                            device = device)
test_iterator = data.BucketIterator(test_data, batch_size=BATCH_SIZE, shuffle=False, sort=False,
                                    device = device)

In [17]:
print("훈련 데이터의 미니 배치의 개수 :", len(train_iterator))
print("검증 데이터의 미니 배치의 개수 :", len(valid_iterator))
print("테스트 데이터의 미니 배치의 개수 :", len(test_iterator))

훈련 데이터의 미니 배치의 개수 : 48
검증 데이터의 미니 배치의 개수 : 12
테스트 데이터의 미니 배치의 개수 : 26


In [18]:
batch = next(iter(train_iterator))
print(batch.text[0])

tensor([  101, 16175, 26869,  4426,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0], device='cuda:0')


In [19]:
train_iterator, valid_iterator  = data.BucketIterator.splits((train_data, valid_data),
                                                            batch_size = BATCH_SIZE, shuffle = True, sort=False,
                                                            device = device)
test_iterator = data.BucketIterator(test_data, batch_size=BATCH_SIZE, shuffle=False, sort=False,
                                    device = device)

## Build Model

In [20]:
bert = BertModel.from_pretrained("bert-base-uncased")

In [22]:
class BERTLSTMSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()  
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.rnn = nn.LSTM(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = True,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout,
                          bias = True)
        self.layernorm = nn.LayerNorm(hidden_dim*2)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
                
        with torch.no_grad():
            embedded = self.bert(text)[0]

        hidden, _ = self.rnn(embedded)
        hidden = hidden[:, -1, :]
        hidden = self.layernorm(hidden)
        output = self.out(hidden)
        return output

In [23]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [24]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

## Train the Model

In [25]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [26]:
model = model.to(device)
criterion = criterion.to(device)

In [27]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [28]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [30]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
N_EPOCHS = 100

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        if not os.path.isdir("pre_trained_bert_ver3"):
            os.makedirs("pre_trained_bert_ver3")
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './pre_trained_bert_ver3/textclassificator.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 0.654 | Train Acc: 62.21%
	 Val. Loss: 0.549 |  Val. Acc: 75.68%
Epoch: 02 | Epoch Time: 0m 9s
	Train Loss: 0.580 | Train Acc: 71.98%
	 Val. Loss: 0.490 |  Val. Acc: 78.85%
Epoch: 03 | Epoch Time: 0m 9s
	Train Loss: 0.474 | Train Acc: 78.88%
	 Val. Loss: 0.422 |  Val. Acc: 80.76%
Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.442 | Train Acc: 81.10%
	 Val. Loss: 0.417 |  Val. Acc: 82.70%
Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.428 | Train Acc: 81.45%
	 Val. Loss: 0.419 |  Val. Acc: 82.25%
Epoch: 06 | Epoch Time: 0m 9s
	Train Loss: 0.411 | Train Acc: 82.23%
	 Val. Loss: 0.420 |  Val. Acc: 82.04%
Epoch: 07 | Epoch Time: 0m 9s
	Train Loss: 0.402 | Train Acc: 82.84%
	 Val. Loss: 0.416 |  Val. Acc: 81.47%
Epoch: 08 | Epoch Time: 0m 9s
	Train Loss: 0.387 | Train Acc: 83.56%
	 Val. Loss: 0.403 |  Val. Acc: 81.96%
Epoch: 09 | Epoch Time: 0m 9s
	Train Loss: 0.375 | Train Acc: 84.05%
	 Val. Loss: 0.423 |  Val. Acc: 83.09%
Epoch: 10 | Epoch Time: 0m 9

Epoch: 77 | Epoch Time: 0m 9s
	Train Loss: 0.037 | Train Acc: 98.20%
	 Val. Loss: 1.126 |  Val. Acc: 79.74%
Epoch: 78 | Epoch Time: 0m 9s
	Train Loss: 0.042 | Train Acc: 98.15%
	 Val. Loss: 0.968 |  Val. Acc: 80.04%
Epoch: 79 | Epoch Time: 0m 9s
	Train Loss: 0.047 | Train Acc: 98.05%
	 Val. Loss: 0.905 |  Val. Acc: 82.28%
Epoch: 80 | Epoch Time: 0m 9s
	Train Loss: 0.037 | Train Acc: 98.21%
	 Val. Loss: 1.158 |  Val. Acc: 80.71%
Epoch: 81 | Epoch Time: 0m 9s
	Train Loss: 0.034 | Train Acc: 98.32%
	 Val. Loss: 1.247 |  Val. Acc: 81.80%
Epoch: 82 | Epoch Time: 0m 9s
	Train Loss: 0.046 | Train Acc: 98.05%
	 Val. Loss: 1.062 |  Val. Acc: 81.64%
Epoch: 83 | Epoch Time: 0m 9s
	Train Loss: 0.038 | Train Acc: 98.23%
	 Val. Loss: 1.153 |  Val. Acc: 80.74%
Epoch: 84 | Epoch Time: 0m 9s
	Train Loss: 0.043 | Train Acc: 98.13%
	 Val. Loss: 1.069 |  Val. Acc: 81.01%
Epoch: 85 | Epoch Time: 0m 9s
	Train Loss: 0.048 | Train Acc: 97.75%
	 Val. Loss: 0.975 |  Val. Acc: 81.89%
Epoch: 86 | Epoch Time: 0m 9

In [32]:
def make_csv(model, iterator):
    
    model.eval()
    result = []
    
    with torch.no_grad():
    
        for batch in iterator:
            x = batch.text
            predictions = model(x).squeeze(0)
            predictions = torch.round(torch.sigmoid(predictions))
            result += predictions
    
    return result
            

In [33]:
model.load_state_dict(torch.load('./pre_trained_bert_ver3/textclassificator.pt'))

<All keys matched successfully>

In [34]:
LABEL.build_vocab(test_data)
print(LABEL.vocab.stoi)
result = make_csv(model, test_iterator)
print(result[:10])

defaultdict(None, {'0': 0})
[tensor([1.], device='cuda:0'), tensor([1.], device='cuda:0'), tensor([1.], device='cuda:0'), tensor([0.], device='cuda:0'), tensor([1.], device='cuda:0'), tensor([1.], device='cuda:0'), tensor([0.], device='cuda:0'), tensor([0.], device='cuda:0'), tensor([0.], device='cuda:0'), tensor([0.], device='cuda:0')]


In [35]:
final_result_csv = pd.DataFrame(result, columns=["target"])
final_result_csv["target"] = final_result_csv["target"].apply(lambda x : int(x.item()))
print(final_result_csv[:10])

   target
0       1
1       1
2       1
3       0
4       1
5       1
6       0
7       0
8       0
9       0


In [36]:
test_csv = pd.read_csv("./train_test_data/test.csv")
test_csv = test_csv.drop(["keyword", "location", "text"], axis="columns")
test_csv.head()

,id
0,0
1,2
2,3
3,9
4,11


In [37]:
final = pd.concat([test_csv, final_result_csv], axis = 1)
final.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [38]:
final.to_csv("./result/pre_trained_bert_ver3.csv", index=False)